# Example from xkcd aka. Subset Sum Problem

Lucerne University of Applied Sciences and Arts - School of Information Technology

@author: Tobias Mérinat

Imports

In [30]:
import math

from ortools.constraint_solver import pywrapcp

[xkcd example](https://xkcd.com/287/)

In [31]:
n = ['fruit', 'fries', 'salad', 'wings', 'sticks', 'plate'] # the items
p = [215, 275, 335, 355, 420, 580] # the weights in subset sum jargon

target_price = 1505 # the capacity in subset sum jargon
item_list=list(zip(n, p))  # make a list because we need to iterate over it twice

#### First, solve Subset Sum xkcd as decision problem and meet pre-defined target price

Create constraint solver

In [32]:
solver = pywrapcp.Solver("Subset Sum xkcd")

Configure variables (the number of times an item is chosen)

In [33]:
choices = []
prices  = [] # decision variables for weights in subset sum jargon
for item_name, item_price in item_list:
    choices.append(solver.IntVar(0, math.floor(target_price/item_price), item_name))
    prices.append(item_price)

Price constraint

In [34]:
solver.Add(solver.ScalProd(choices, prices) == target_price)

Configure solver

In [35]:
db = solver.Phase(choices, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [48]:
solver.NewSearch(db)
while solver.NextSolution():
    for item, price in zip(choices, prices):
        print("Item {} with value {:.2f} has been chosen {} times".format(item.Name(), price/100, item.Value()))
    print("\n")

Item fruit with value 2.15 has been chosen 0 times
Item fries with value 2.75 has been chosen 0 times
Item salad with value 3.35 has been chosen 0 times
Item wings with value 3.55 has been chosen 0 times
Item sticks with value 4.20 has been chosen 0 times
Item plate with value 5.80 has been chosen 0 times


Item fruit with value 2.15 has been chosen 0 times
Item fries with value 2.75 has been chosen 0 times
Item salad with value 3.35 has been chosen 0 times
Item wings with value 3.55 has been chosen 0 times
Item sticks with value 4.20 has been chosen 0 times
Item plate with value 5.80 has been chosen 1 times


Item fruit with value 2.15 has been chosen 0 times
Item fries with value 2.75 has been chosen 0 times
Item salad with value 3.35 has been chosen 0 times
Item wings with value 3.55 has been chosen 0 times
Item sticks with value 4.20 has been chosen 0 times
Item plate with value 5.80 has been chosen 2 times


Item fruit with value 2.15 has been chosen 0 times
Item fries with value 

Cleanup

In [37]:
solver.EndSearch()

Print solver information

In [38]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 2
Runtime:   29ms
Failures:  79
Branches:  160 


#### Next, solve Subset Sum xkcd as optimization problem and approach pre-defined target price

Create constraint solver

In [39]:
solver = pywrapcp.Solver("Subset Sum xkcd optimization problem")

Configure variables (the number of times an item is chosen)

In [40]:
# Note: we cannot reuse the same decision variables as above as they have already been processed by the solver
choices = []
prices  = [] 
for item_name, item_price in item_list:
    choices.append(solver.IntVar(0, math.floor(target_price/item_price), item_name))
    prices.append(item_price)

Price must not exceed target price ("weights must not exceed "capacity")

In [41]:
total = solver.ScalProd(choices, prices).Var()
approx = (target_price - total).Var()
solver.Add(approx >= 0)

Price objective (solver.Difference is not available in the Python wrapper)

In [42]:
objective = solver.Minimize(approx, 1)

Configure solver

In [43]:
db = solver.Phase(choices, solver.INT_VAR_SIMPLE, solver.INT_VALUE_SIMPLE)

Start solver

In [44]:
solver.NewSearch(db, objective)
while solver.NextSolution():
    print("Total Price: {:.2f} / Upper Bound: {:.2f}".format(total.Value()/100, target_price/100))
    for item, price in zip(choices, prices):
        print(" - Item {} with value {:.2f} has been chosen {} times".format(item.Name(), price/100, item.Value()))
    print("\n")

Total Price: 0.00 / Upper Bound: 15.05
 - Item fruit with value 2.15 has been chosen 0 times
 - Item fries with value 2.75 has been chosen 0 times
 - Item salad with value 3.35 has been chosen 0 times
 - Item wings with value 3.55 has been chosen 0 times
 - Item sticks with value 4.20 has been chosen 0 times
 - Item plate with value 5.80 has been chosen 0 times


Total Price: 5.80 / Upper Bound: 15.05
 - Item fruit with value 2.15 has been chosen 0 times
 - Item fries with value 2.75 has been chosen 0 times
 - Item salad with value 3.35 has been chosen 0 times
 - Item wings with value 3.55 has been chosen 0 times
 - Item sticks with value 4.20 has been chosen 0 times
 - Item plate with value 5.80 has been chosen 1 times


Total Price: 11.60 / Upper Bound: 15.05
 - Item fruit with value 2.15 has been chosen 0 times
 - Item fries with value 2.75 has been chosen 0 times
 - Item salad with value 3.35 has been chosen 0 times
 - Item wings with value 3.55 has been chosen 0 times
 - Item stic

Cleanup

In [45]:
solver.EndSearch()

Print solver information

In [46]:
print("Solutions: {}".format(solver.Solutions()))
print("Runtime:   {}ms".format(solver.WallTime()))
print("Failures:  {}".format(solver.Failures()))
print("Branches:  {} ".format(solver.Branches()))

Solutions: 7
Runtime:   38ms
Failures:  33
Branches:  73 
